# Lists of S&P 500 companies historical components

1. Read 'S&P 500 Historical Components & Changes.csv' which contains historical S&P 500 index membership since 1996.  Each row contains 2 fields: date and tickers; tickers is a comma delimited string of the symbols for that specified date.  This file was provided as open source via download for the book "Trading Evolved" by Andreas F. Clenow.  
2. Clean up the list by fixing up symbols, removing duplicates in each row, and sorting the symbols  
3. Read in 'sp500_changes_since_2019.csv' which contains the changes to the index since 2019  
4. Create new rows in the historical membership list for these changes  
5. Compare the last row to current list of S&P 500 components on the wikipedia  
6. A list of differences is generated.  If this list is NOT empty, then changes have been made to the index that should be included in 'sp500_changes_since_2019.csv'.  
7. Write out date stamped 'S&P 500 Historical Components & Changes' csv file

In [18]:
from datetime import datetime
import os
import pandas as pd

pd.options.mode.chained_assignment = None  # default='warn'
pd.set_option('display.max_rows', 600)

# -*- encoding: utf-8 -*-
%matplotlib inline

In [19]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [20]:
def get_table(filename):

    if os.path.isfile(filename):
        df = pd.read_csv(filename, index_col='date')
        return df

In [21]:
filename = 'S&P 500 Historical Components & Changes.csv'
df = get_table(filename)
df.head()

,tickers
date,
1996-01-02,"TMC-200006,AAL-199702,AAMRQ-201312,AAPL,ABI-20..."
1996-01-03,"AAL-199702,AAMRQ-201312,AAPL,ABI-200811,ABS-20..."
1996-01-04,"TMC-200006,AAL-199702,AAMRQ-201312,AAPL,OAT-20..."
1996-01-10,"TMC-200006,AAL-199702,AAMRQ-201312,AAPL,ABI-20..."
1996-01-11,"FLMIQ-200408,FLTWQ-200907,FMC,FMCC,FNMA,FTL.A-..."


In [22]:
# Convert ticker column from csv to list, then sort.
df['tickers'] = df['tickers'].apply(lambda x: sorted(x.split(',')))
df.tail()

,tickers
date,
2019-01-03,"[A, AAL, AAP, AAPL, ABBV, ABC, ABMD, ABT, ACN,..."
2019-01-08,"[A, AAL, AAP, AAPL, ABBV, ABC, ABMD, ABT, ACN,..."
2019-01-09,"[A, AAL, AAP, AAPL, ABBV, ABC, ABMD, ABT, ACN,..."
2019-01-10,"[A, AAL, AAP, AAPL, ABBV, ABC, ABMD, ABT, ACN,..."
2019-01-11,"[A, AAL, AAP, AAPL, ABBV, ABC, ABMD, ABT, ACN,..."


In [23]:
l = list(df['tickers'].head(100))[0]
l

['AAL-199702',
 'AAMRQ-201312',
 'AAPL',
 'ABI-200811',
 'ABS-200606',
 'ABT',
 'ABX',
 'ACKH-200211',
 'ACV-201105',
 'ADM',
 'ADP',
 'ADSK',
 'AEE',
 'AEP',
 'AET',
 'AGC-200108',
 'AGN-201503',
 'AHM-199810',
 'AIG',
 'AIT-199910',
 'AL-200711',
 'ALL',
 'AM-201308',
 'AMAT',
 'AMD',
 'AMGN',
 'AMH-199709',
 'AMP-199904',
 'AN-199812',
 'ANDW-200712',
 'ANV-199904',
 'APD',
 'AR-199911',
 'ARC-200004',
 'ARNC',
 'AS-199909',
 'ASC-199906',
 'ASH',
 'AT-200711',
 'ATI',
 'ATI-199906',
 'AVP',
 'AVY',
 'AXP',
 'AZA.A-200106',
 'BA',
 'BAC',
 'BAC-199809',
 'BAX',
 'BBI-199801',
 'BC',
 'BCO',
 'BCR-201712',
 'BDK-201003',
 'BDX',
 'BEAM-201404',
 'BEV-200603',
 'BF.B',
 'BFI-199907',
 'BFO-200010',
 'BGG',
 'BHGE',
 'BHMSQ-200401',
 'BK',
 'BKB-199909',
 'BLL',
 'BLS-200612',
 'BLY-199612',
 'BMET-200709',
 'BMS',
 'BMY',
 'BNI-201002',
 'BNL-199806',
 'BOAT-199701',
 'BOL-200710',
 'BR-200603',
 'BSX',
 'BT-199906',
 'BUD-200811',
 'C',
 'C-199811',
 'CA',
 'CAG',
 'CAL',
 'CAR',
 'C

In [24]:
# Replace SYMBOL-yyyymm with SYMBOL.
df['tickers'] = [[ticker.split('-')[0] for ticker in tickers] for tickers in df['tickers']]
df.head()

,tickers
date,
1996-01-02,"[AAL, AAMRQ, AAPL, ABI, ABS, ABT, ABX, ACKH, A..."
1996-01-03,"[AAL, AAMRQ, AAPL, ABI, ABS, ABT, ABX, ACKH, A..."
1996-01-04,"[AAL, AAMRQ, AAPL, ABI, ABS, ABT, ABX, ACKH, A..."
1996-01-10,"[AAL, AAMRQ, AAPL, ABI, ABS, ABT, ABX, ACKH, A..."
1996-01-11,"[AAL, AAMRQ, AAPL, ABI, ABS, ABT, ABX, ACKH, A..."


In [25]:
l = list(df['tickers'].tail(1))[0]
len(l)

504

In [26]:
# Add LIN after 2018-10-31
df.loc[df.index > '2018-10-31', 'tickers'].apply(lambda x: x.append('LIN'))
'LIN' in df['tickers'].values[-1]

True

In [27]:
# Remove duplicates in each row.
df['tickers'] = [sorted(list(set(tickers))) for tickers in df['tickers']]
df.tail()
num = []
for i, row in df.iterrows():
    l = list(row['tickers'])
    num.append(len(l))
    print(i, len(l))
pd.Series(num).describe()

1996-01-02 487
1996-01-03 487
1996-01-04 487
1996-01-10 487
1996-01-11 487
1996-01-12 487
1996-01-22 487
1996-01-23 487
1996-01-24 487
1996-01-30 487
1996-01-31 487
1996-02-01 487
1996-02-12 487
1996-02-13 487
1996-02-15 487
1996-02-20 487
1996-02-21 487
1996-02-22 487
1996-03-05 487
1996-03-08 487
1996-03-11 487
1996-03-12 487
1996-03-13 487
1996-03-14 487
1996-03-15 487
1996-03-25 487
1996-03-28 487
1996-03-29 487
1996-04-01 487
1996-04-02 487
1996-04-03 487
1996-04-04 487
1996-04-15 487
1996-04-19 487
1996-04-22 487
1996-04-23 487
1996-04-24 487
1996-04-25 487
1996-05-03 487
1996-05-09 487
1996-05-10 487
1996-05-13 487
1996-05-23 487
1996-05-30 487
1996-05-31 487
1996-06-03 487
1996-06-04 487
1996-06-05 487
1996-06-20 487
1996-06-26 487
1996-06-27 487
1996-06-28 487
1996-07-11 487
1996-07-17 487
1996-07-18 487
1996-07-19 487
1996-07-22 487
1996-07-23 487
1996-07-31 487
1996-08-06 487
1996-08-07 487
1996-08-08 487
1996-08-16 488
1996-08-19 488
1996-08-20 488
1996-08-26 488
1996-08-27

2004-12-21 495
2004-12-22 495
2004-12-23 495
2004-12-27 495
2004-12-29 495
2004-12-30 495
2005-01-06 495
2005-01-12 495
2005-01-13 495
2005-01-14 495
2005-01-27 495
2005-02-02 495
2005-02-03 495
2005-02-04 495
2005-02-16 495
2005-02-23 495
2005-02-24 495
2005-02-25 495
2005-03-09 495
2005-03-14 495
2005-03-15 495
2005-03-16 495
2005-03-17 495
2005-03-21 495
2005-03-28 496
2005-03-29 496
2005-03-30 496
2005-04-05 496
2005-04-06 496
2005-04-07 496
2005-04-19 496
2005-04-25 496
2005-04-26 496
2005-04-27 496
2005-05-09 496
2005-05-13 496
2005-05-16 496
2005-05-17 496
2005-05-27 496
2005-06-03 496
2005-06-06 496
2005-06-10 496
2005-06-16 496
2005-06-17 496
2005-06-20 496
2005-06-30 496
2005-07-05 496
2005-07-06 496
2005-07-07 496
2005-07-08 496
2005-07-11 496
2005-07-21 496
2005-07-22 496
2005-07-25 496
2005-07-27 496
2005-07-28 496
2005-07-29 496
2005-08-10 496
2005-08-11 496
2005-08-12 496
2005-08-15 496
2005-08-16 496
2005-08-17 496
2005-08-18 496
2005-08-19 496
2005-08-22 496
2005-08-29

2010-03-10 499
2010-03-11 499
2010-03-15 499
2010-03-16 499
2010-03-23 499
2010-03-29 499
2010-03-30 499
2010-03-31 499
2010-04-13 499
2010-04-19 499
2010-04-20 499
2010-04-21 499
2010-04-29 498
2010-04-30 499
2010-05-03 499
2010-05-05 499
2010-05-07 499
2010-05-10 499
2010-05-11 499
2010-05-21 499
2010-05-27 499
2010-05-28 499
2010-06-01 499
2010-06-11 499
2010-06-17 499
2010-06-18 499
2010-06-21 499
2010-06-28 499
2010-06-29 499
2010-07-01 499
2010-07-02 499
2010-07-06 499
2010-07-08 499
2010-07-09 499
2010-07-12 499
2010-07-15 498
2010-07-16 498
2010-07-22 498
2010-07-28 498
2010-07-29 498
2010-07-30 498
2010-08-11 498
2010-08-17 498
2010-08-18 498
2010-08-19 498
2010-08-27 497
2010-08-30 497
2010-08-31 497
2010-09-07 497
2010-09-08 497
2010-09-09 497
2010-09-21 497
2010-09-27 497
2010-09-28 497
2010-09-29 497
2010-10-11 497
2010-10-15 497
2010-10-18 497
2010-10-19 497
2010-10-29 497
2010-11-04 497
2010-11-05 497
2010-11-08 497
2010-11-17 497
2010-11-18 497
2010-11-22 497
2010-11-24

2016-06-03 505
2016-06-06 505
2016-06-07 505
2016-06-08 505
2016-06-09 505
2016-06-20 505
2016-06-21 504
2016-06-22 504
2016-06-23 504
2016-06-24 505
2016-06-27 505
2016-07-01 504
2016-07-05 505
2016-07-06 505
2016-07-07 505
2016-07-11 505
2016-07-12 505
2016-07-13 505
2016-07-25 505
2016-07-29 505
2016-08-01 505
2016-08-02 505
2016-08-05 505
2016-08-11 505
2016-08-12 505
2016-08-15 505
2016-08-25 505
2016-08-31 505
2016-09-01 505
2016-09-02 505
2016-09-06 506
2016-09-07 505
2016-09-08 506
2016-09-09 506
2016-09-14 506
2016-09-15 506
2016-09-16 506
2016-09-23 506
2016-09-26 506
2016-09-28 506
2016-10-03 506
2016-10-04 506
2016-10-05 506
2016-10-06 506
2016-10-10 506
2016-10-18 506
2016-10-24 506
2016-10-25 506
2016-10-26 506
2016-10-31 506
2016-11-04 506
2016-11-07 506
2016-11-08 506
2016-11-18 506
2016-11-25 506
2016-11-28 506
2016-11-29 506
2016-12-02 506
2016-12-05 506
2016-12-09 506
2016-12-12 506
2016-12-22 506
2016-12-29 506
2016-12-30 506
2017-01-03 506
2017-01-05 506
2017-01-06

count    2595.000000
mean      496.439306
std         5.041279
min       487.000000
25%       494.000000
50%       497.000000
75%       499.000000
max       507.000000
dtype: float64

In [28]:
l = list(df['tickers'].head(3100))[1500]
len(l)

498

Changes to the list of S&P 500 components
https://en.wikipedia.org/wiki/List_of_S%26P_500_companies#Selected_changes_to_the_list_of_S&P_500_components

In [29]:
# Read S&P 500 changes csv file.
filename = 'sp500_changes_since_2019.csv'
changes = get_table(filename)

# Convert ticker column from csv to list, then sort.
changes['add'] = changes['add'].apply(lambda x: sorted(x.split(',')))
changes['remove'] = changes['remove'].apply(lambda x: sorted(x.split(',')))
changes

,add,remove
date,,
2019-01-18,[TFX],[PCG]
2019-02-15,[ATO],[NFX]
2019-02-27,[WAB],[GT]
2019-04-02,[DOW],[BHF]
2019-06-01,[LHX],[HRS]
2019-06-03,"[CTVA, DD]","[DWDP, FLR]"
2019-06-07,[AMCR],[MAT]
2019-07-01,[MKTX],[LLL]
2019-07-15,[TMUS],[RHT]


In [30]:
# Copy the last row in dataframe, modify for changes, then append.
for change in changes.itertuples():

    new_row = df.tail(1)
    
    tickers = list(new_row['tickers'][0])
    tickers += change.add
    tickers = list(set(tickers) - set(change.remove))
    tickers = sorted(tickers)
    
    d = {'date':change.Index, 'tickers':[tickers]}
    new_entry = pd.DataFrame(d)
    new_entry.set_index('date', inplace=True)
    df = df.append(new_entry)

In [31]:
df.tail()

,tickers
date,
2021-04-20,"[A, AAL, AAP, AAPL, ABBV, ABC, ABMD, ABT, ACN,..."
2021-05-14,"[A, AAL, AAP, AAPL, ABBV, ABC, ABMD, ABT, ACN,..."
2021-06-04,"[A, AAL, AAP, AAPL, ABBV, ABC, ABMD, ABT, ACN,..."
2021-07-21,"[A, AAL, AAP, AAPL, ABBV, ABC, ABMD, ABT, ACN,..."
2021-08-03,"[A, AAL, AAP, AAPL, ABBV, ABC, ABMD, ABT, ACN,..."


In [32]:
# compare last row to current S&P500 list
filename = 'sp500.csv'
current = pd.read_csv(filename)
current = list(current['Symbol'])
last_entry = list(df['tickers'][-1])

diff = list(set(current) - set(last_entry)) + list(set(last_entry) - set(current))
diff

['BRK-B', 'BRK.B']

In [33]:
# Convert tickers column back to csv.
df['tickers'] = df['tickers'].apply(lambda x: ",".join(x))
df.head()

,tickers
date,
1996-01-02,"AAL,AAMRQ,AAPL,ABI,ABS,ABT,ABX,ACKH,ACV,ADM,AD..."
1996-01-03,"AAL,AAMRQ,AAPL,ABI,ABS,ABT,ABX,ACKH,ACV,ADM,AD..."
1996-01-04,"AAL,AAMRQ,AAPL,ABI,ABS,ABT,ABX,ACKH,ACV,ADM,AD..."
1996-01-10,"AAL,AAMRQ,AAPL,ABI,ABS,ABT,ABX,ACKH,ACV,ADM,AD..."
1996-01-11,"AAL,AAMRQ,AAPL,ABI,ABS,ABT,ABX,ACKH,ACV,ADM,AD..."


In [34]:
now = datetime.now()
dt_string = now.strftime('%m-%d-%Y') # mm-dd-YYYY
filename = 'S&P 500 Historical Components & Changes({}).csv'.format(dt_string)
df.to_csv(filename)